Задание 1. Weighted Random Recommendation Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

Можно сэмплировать товары случайно, но пропорционально какому-либо весу Например, прямопропорционально популярности. вес = log(sales_sum товара) Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
def weighted_random_recommendation(items_weights, n=5):
  
    item_id = np.array(items_weights.item_id)
   
    sum_=items_weights['weight'].sum()
    
    items_weights['pp']  = items_weights.apply(lambda row:row['weight']/sum_,axis = 1)
     
    recs = np.random.choice(item_id, size=n, replace=False,p = items_weights['pp'] )
        
    return recs.tolist()

In [5]:
#на основании sales_value

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()

items_weights['weight'] = items_weights.apply(lambda row: log(row['sales_value'] if row['sales_value'] !=0 else 1) 
                                              if log(row['sales_value'] if row['sales_value'] !=0 else 1)>=0 else 0,
                                              axis = 1)

items_weights.head(3)

,item_id,sales_value,weight
0,25671,20.94,3.041661
1,26081,0.99,0.000000
2,26093,1.59,0.463734


In [6]:
%%time
result = weighted_random_recommendation(items_weights)
print(result)

[896513, 13133879, 1086274, 866548, 13911532]
Wall time: 1.28 s


In [7]:
#на основании цены

items_weights = data_train.groupby('item_id')['quantity','sales_value'].sum().reset_index()

items_weights['weight'] = items_weights.apply(lambda row: row['sales_value']/row['quantity'] if row['quantity']>0 else 0, axis=1  )
items_weights.head(3)

C:\Users\Yulia\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,item_id,quantity,sales_value,weight
0,25671,6,20.94,3.49
1,26081,1,0.99,0.99
2,26093,1,1.59,1.59


In [8]:
%%time
result = weighted_random_recommendation(items_weights)
print(result)

[1386987, 1118303, 9297456, 1041494, 861732]
Wall time: 1.14 s


In [9]:
#что больше продавалось относительно недавно
items_weights = data_train.groupby(['item_id','week_no'])['quantity'].count().reset_index()
items_weights.head(3)
items_weights['weight'] = items_weights['week_no']*items_weights['quantity']
items_weights = items_weights.groupby(['item_id'])['weight'].max().reset_index()
items_weights.head(3)

,item_id,weight
0,25671,59
1,26081,36
2,26093,66


In [10]:
%%time
result = weighted_random_recommendation(items_weights)
print(result)

[889863, 838824, 915617, 962474, 5583944]
Wall time: 1.08 s
